# **Movie Recommendation System**

 # **1-) Libraries**


In [ ]:
!pip install flask pyngrok pandas numpy scikit-learn

In [ ]:
from flask import Flask, request, render_template_string
from pyngrok import ngrok
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import logging

logging.basicConfig(filename='recommendation_logs.log', level=logging.INFO, format='%(asctime)s - %(message)s')


# **2-) Datasets**

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

movies = movies.merge(credits, left_on='id', right_on='movie_id', how='inner')

movies = movies[['title_x', 'genres', 'keywords', 'overview', 'crew']]

movies.rename(columns={'title_x': 'title'}, inplace=True)

def parse_list_field(field):
    try:
        data = ast.literal_eval(field)
        return " ".join([d['name'].replace(" ", "") for d in data])
    except:
        return ""

def get_director(crew_str):
    try:
        data = ast.literal_eval(crew_str)
        for c in data:
            if c['job'] == 'Director':
                return c['name'].replace(" ", "")
        return ""
    except:
        return ""

movies['genres'] = movies['genres'].apply(parse_list_field)
movies['keywords'] = movies['keywords'].apply(parse_list_field)
movies['director'] = movies['crew'].apply(get_director)
movies.drop('crew', axis=1, inplace=True)

movies['combined_features'] = (
    movies['genres'] + " " +
    movies['keywords'] + " " +
    movies['overview'] + " " +
    movies['director']
)

movies.dropna(subset=['combined_features'], inplace=True)

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['combined_features'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

title_to_index = pd.Series(movies.index, index=movies['title']).drop_duplicates()


# **3-) Recommendation Functions**

In [ ]:
def recommend_movies_for_one(movie_title, num_recs=5):
    if movie_title not in title_to_index:
        return [f"Movie '{movie_title}' not found in the dataset!"]

    idx = title_to_index[movie_title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_recs + 1]
    movie_indices = [i[0] for i in sim_scores]
    return movies['title'].iloc[movie_indices].tolist()


In [ ]:
def recommend_movies_for_multiple(favorite_movies, num_recs=5):
    valid_favorites = [m for m in favorite_movies if m in title_to_index]
    if not valid_favorites:
        return ["No valid movies found in the dataset!"]

    sim_scores_sum = np.zeros(cosine_sim.shape[0])
    for movie in valid_favorites:
        idx = title_to_index[movie]
        sim_scores_sum += cosine_sim[idx]

    sim_scores_avg = sim_scores_sum / len(valid_favorites)
    sim_scores_avg = list(enumerate(sim_scores_avg))
    sim_scores_avg = sorted(sim_scores_avg, key=lambda x: x[1], reverse=True)

    fav_indices = [title_to_index[m] for m in valid_favorites]
    sim_scores_avg = [s for s in sim_scores_avg if s[0] not in fav_indices]

    sim_scores_avg = sim_scores_avg[:num_recs]
    movie_indices = [i[0] for i in sim_scores_avg]
    return movies['title'].iloc[movie_indices].tolist()


# **4-) Flask Integration with UI**

In [ ]:
html_template = """
<!DOCTYPE html>
<html>
<head>
  <title>Stylish Movie Recommendation System</title>
  <style>
    body {
      font-family: 'Arial', sans-serif;
      background-color: #f4f4f4;
      margin: 0;
      padding: 0;
    }
    .container {
      max-width: 800px;
      margin: 50px auto;
      background: #ffffff;
      padding: 30px;
      box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
      border-radius: 8px;
    }
    h1 {
      color: #333;
      text-align: center;
    }
    form {
      margin-top: 20px;
      text-align: center;
    }
    input[type="text"], input[type="number"] {
      padding: 10px;
      width: 70%;
      margin-bottom: 15px;
      border: 1px solid #ddd;
      border-radius: 5px;
    }
    input[type="submit"] {
      padding: 10px 20px;
      background-color: #28a745;
      color: #fff;
      border: none;
      border-radius: 5px;
      cursor: pointer;
      font-size: 16px;
    }
    input[type="submit"]:hover {
      background-color: #218838;
    }
    ul {
      list-style: none;
      padding: 0;
    }
    li {
      background: #f9f9f9;
      margin: 5px 0;
      padding: 10px;
      border: 1px solid #ddd;
      border-radius: 5px;
    }
  </style>
  <script>
    function validateForm() {
      const movies = document.forms["recommendForm"]["movies"].value;
      if (movies.trim() === "") {
        alert("Please enter at least one movie!");
        return false;
      }
      return true;
    }
  </script>
</head>
<body>
  <div class="container">
    <h1>Movie Recommendation System</h1>
    <form name="recommendForm" method="POST" action="/recommend" onsubmit="return validateForm()">
      <input type="text" name="movies" placeholder="Enter your favorite movies (comma-separated)" required>
      <input type="number" name="num_recs" value="5" min="1" max="20" required>
      <input type="submit" value="Get Recommendations">
    </form>
  </div>
</body>
</html>
"""


# 5-) **Flask App Logic**

In [ ]:
app = Flask(__name__)

@app.route('/', methods=['GET'])
def index():
    return render_template_string(html_template)

@app.route('/recommend', methods=['POST'])
def recommend():
    favs = request.form['movies']
    num = int(request.form['num_recs'])
    fav_list = [m.strip() for m in favs.split(',')]
    recs = recommend_movies_for_multiple(fav_list, num)

    logging.info(f"User input: {favs} | Recommendations: {recs}")

    result_html = f"""
    <div class="container">
      <h1>Recommendations</h1>
      <ul>
        {"".join(f"<li>{r}</li>" for r in recs)}
      </ul>
      <a href="/">Go back</a>
    </div>
    """
    return result_html


# **6-) Flask App with ngrok**

In [ ]:
!ngrok config add-authtoken 2q0BRuID0aPcAT3eNzDnSAr4Jo2_39W7cZ1tHUuQpYUfyvj9q

In [ ]:
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

app.run(port=5000)
